# Variational Autoencoder 

Autoencoder (AE) is a method to map high-dimensional signal into lower-dimensional latent space [1]. However, for signal creation it is not strightforward to sample a low-dimensional signal from latent space, and generate a high-dimensional signal that follows the distribution of training data.  

Different from AE which maps the input to a vector, variatoinal autoencoder (VAE) restricts the latent representation only as a distribution. There are a few advantages for such setting:  

## Autoencoder 

In [1]:
import numpy as np
import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

datapath = r'C:\Users\User\Documents\repos\data'

In [2]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128), 
            nn.ReLU(inplace=True), 
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12), 
            nn.ReLU(True),
            nn.Linear(12, 3)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128), 
            nn.ReLU(True), 
            nn.Linear(128, 28*28), 
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
        

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(datapath, train=True, transform=transform, download=False)
test_dataset = datasets.MNIST(datapath, train=False, transform=transform, download=False)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [4]:
model = AutoEncoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
num_epochs = 2

for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        #print('imgsize', img.size(0), img[0].shape, img.shape)
        img = img.reshape(img.size(0), -1)
        #print(img.shape)
        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

## Evidence Lower Bound 


The Evidence Lower Bound (ELBO) is a concept primarily used in the context of variational inference, a method for approximating complex probability distributions. In probabilistic modeling, we often want to infer the posterior distribution of latent variables given observed data. However, calculating the true posterior distribution is often unfeasible. Variational inference approximates the posterior distribution with a simpler distribution chosen from a parameterized function, such as a Gaussian distribution. 

The ELBO serves as a lower bound for the log marginal likelihood of the data. By maximizing the ELBO with respect to the parameters of the approximate posterior distribution, we indirectly maximize the log marginal likelihood. This is because the ELBO is derived from the Kullback-Leibler (KL) divergence between the approximate posterior and the true posterior, and maximizing the ELBO minimizes this divergence.

## Varitaional Autoencoder 

## Reference 

- https://kvfrans.com/variational-autoencoders-explained/